In [ ]:
#estas librerias sirven para el procesamiento de los datos
import os
import pandas as pd
from pathlib import Path

def extract_and_combine_sheets(directory, sheet_name, output_directory):
    extracted_data = []
    total_records = 0  # Inicializo un contador para los registros que voy a procesar.
    Path(output_directory).mkdir(parents=True, exist_ok=True)  # Me aseguro de que el directorio de salida exista o lo creo.

    files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.xlsx')]  # Creo una lista de todos los archivos Excel en el directorio especificado.

    for file in files:
        try:
            data = pd.read_excel(file, sheet_name=sheet_name, engine='openpyxl')  # Cargar la hoja especificada de cada archivo usando pandas.
            for col in data.columns:  # Elimino las fórmulas de las celdas convirtiéndolas a sus valores.
                data[col] = data[col].apply(lambda x: x.value if hasattr(x, 'value') else x)
            extracted_data.append(data)  # Agrego el dataframe a la lista de dataframes extraídos.
            num_records = data.shape[0]  # Cuento los registros extraídos.
            total_records += num_records  # Acumulo el conteo de registros.
            print(f"Extraídos {num_records} registros de '{sheet_name}' en '{file}'")
            output_file = os.path.join(output_directory, os.path.basename(file))  # Defino el nombre del archivo de salida.
            data.to_excel(output_file, index=False)  # Guardo los datos extraídos en un nuevo archivo Excel.
            print(f"Datos de '{sheet_name}' en '{file}' guardados en '{output_file}'")
        except Exception as e:
            print(f"Error extrayendo de {file}: {str(e)}")  # Imprimo errores si ocurren durante la extracción.

    combined_data = pd.concat(extracted_data, ignore_index=True)  # Combino todos los dataframes en uno solo.
    combined_output_file = os.path.join(output_directory, "archivo_combinado.xlsx")  # Defino el nombre del archivo final combinado.
    combined_data.to_excel(combined_output_file, index=False)  # Guardo el dataframe combinado en un archivo Excel.
    print(f"Total de registros extraídos de todos los archivos: {total_records}")  # Imprimo el total de registros procesados.
    print(f"Datos combinados y guardados en {combined_output_file}")


directory = "/content/workinhouse" # aqui subimos la ruta
sheet_name = "inspecciones app" # la hoja que queremos transformar
output_directory = "/content/workinhouse/outputs" # aqui se genra la ruta del nuevo archivo

extract_and_combine_sheets(directory, sheet_name, output_directory)  # Llamo a la función con los parámetros adecuados.
